import

In [ ]:
# === Cell 1: Imports ===

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
import hopsworks
import config


In [ ]:
import os
os.environ["HF_API_KEY"] = ""

In [ ]:
# === Cell 2: Connect to Hopsworks ===

from config import HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        # project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )
fs = project.get_feature_store()


In [ ]:
# === Cell 3: Load Feature Views ===

metadata_fv = fs.get_feature_view(
    name="paper_metadata_fv",
    version=1,
)

chunk_fv = fs.get_feature_view(
    name="paper_chunk_fv",
    version=1,
)

metadata_fv.init_serving(1)
chunk_fv.init_serving(1)


In [ ]:
# === Cell 4: Load Embedding Model ===
sentence_transformer = SentenceTransformer(
    config.EMBEDDING_MODEL_NAME
)

In [ ]:
# === Cell 5: Similarity Search Engine ===

from functions.similarity_search import SimilaritySearchEngine

search_engine = SimilaritySearchEngine(
    embedding_model=sentence_transformer,
    metadata_feature_view=metadata_fv,
    chunk_feature_view=chunk_fv,
)


In [ ]:
# === Cell 6: Context Builder ===

from functions.context_builder import ContextBuilder

context_builder = ContextBuilder(
    max_tokens=2000,
    max_chunks=8,
)


In [ ]:
# === Cell 7: Prompt Synthesizer ===

from functions.prompt_synthesis import PromptSynthesizer

prompt_synthesizer = PromptSynthesizer(
    system_instruction=(
        "You are a research assistant. "
        "Answer questions based only on provided evidence."
    ),
    agent_policy=(
        "If evidence is insufficient, explicitly say so. "
        "Do not hallucinate."
    ),
)


In [ ]:
# === Cell 8: MCP Dispatcher ===

from functions.mcp_dispatcher import MCPDispatcher

mcp_dispatcher = MCPDispatcher(
    search_engine=search_engine
)


In [ ]:
# === Cell 9: Agentic Inference ===

from functions.agent_loop import AgenticInference
from functions.llm_wrapper import LLMWrapper  

llm = LLMWrapper(
    model_name_or_path="mistralai/Mistral-7B-v0.1"  
)

agent = AgenticInference(
    llm=llm,
    search_engine=search_engine,
    context_builder=context_builder,
    prompt_synthesizer=prompt_synthesizer,
    mcp_dispatcher=mcp_dispatcher,
)


In [ ]:
# === Cell 10: Run Agent ===

query = "What are the best risk reporting practices?"

answer = agent.run(query)
print(answer)
